In [7]:
get_ipython().system('pip install pandas numpy scikit-learn')

Defaulting to user installation because normal site-packages is not writeable


In [8]:
# Importing necessary libraries
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
import numpy as np

In [18]:
import pandas as pd

# Define file paths as strings
transactions_file = "D:\\RAJAT\\Recommnder\\Transactions.csv"
customer_file = "D:\\RAJAT\\Recommnder\\Customer.csv"
prod_cat_file = "D:\\RAJAT\\Recommnder\\prod_cat_info.csv"

# Read the CSV files into DataFrames
transactions_df = pd.read_csv(transactions_file)
customer_df = pd.read_csv(customer_file)
prod_cat_info_df = pd.read_csv(prod_cat_file)


In [19]:
# Merging the dataframes
merged_df = pd.merge(transactions_df, prod_cat_info_df, how='left', left_on=['prod_cat_code', 'prod_subcat_code'], right_on=['prod_cat_code', 'prod_sub_cat_code'])
final_df = pd.merge(merged_df, customer_df, how='left', left_on='cust_id', right_on='customer_Id')


In [20]:

# Handling missing values
final_df['Gender'].fillna(final_df['Gender'].mode()[0], inplace=True)
final_df['city_code'].fillna(final_df['city_code'].mode()[0], inplace=True)

In [21]:
# Converting 'tran_date' and 'DOB' to datetime
final_df['tran_date'] = pd.to_datetime(final_df['tran_date'], format='%d-%m-%Y', errors='coerce')
final_df['DOB'] = pd.to_datetime(final_df['DOB'], format='%d-%m-%Y', errors='coerce')



In [22]:
# Feature Engineering
current_year = pd.Timestamp.now().year
final_df['age'] = current_year - final_df['DOB'].dt.year
customer_spending = final_df.groupby('cust_id')['total_amt'].sum().reset_index()
customer_spending.columns = ['cust_id', 'total_spent']
final_df = pd.merge(final_df, customer_spending, on='cust_id', how='left')

In [23]:

# Collaborative Filtering using TruncatedSVD
user_item_matrix = final_df.pivot_table(index='cust_id', columns='prod_cat_code', values='total_amt', fill_value=0)


In [24]:
# Checking the number of features (columns) in the user-item matrix
num_features = user_item_matrix.shape[1]
print(f'Number of features (columns) in the user-item matrix: {num_features}')


Number of features (columns) in the user-item matrix: 6


In [25]:
# Set n_components to a value less than or equal to the number of features
n_components = min(20, num_features)
print(f'Number of components for TruncatedSVD: {n_components}')

Number of components for TruncatedSVD: 6


In [26]:
# Applying matrix factorization
svd = TruncatedSVD(n_components=n_components, random_state=42)
matrix = svd.fit_transform(user_item_matrix)
reconstructed_matrix = np.dot(matrix, svd.components_)


In [27]:
# Evaluating the model using RMSE
mse = mean_squared_error(user_item_matrix, reconstructed_matrix)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 9.21586742985565e-13


In [28]:
# Function to generate recommendations
def get_top_n_recommendations(user_id, user_item_matrix, reconstructed_matrix, n=10):
    user_idx = user_item_matrix.index.get_loc(user_id)
    user_ratings = reconstructed_matrix[user_idx]
    top_indices = user_ratings.argsort()[::-1][:n]
    top_items = user_item_matrix.columns[top_indices]
    return top_items


In [29]:
user_id = 270351
recommendations = get_top_n_recommendations(user_id, user_item_matrix, reconstructed_matrix)
print("Top N Recommendations for user_id {}: {}".format(user_id, recommendations))

Top N Recommendations for user_id 270351: Index([3, 2, 5, 4, 6, 1], dtype='int64', name='prod_cat_code')


In [30]:
# Mapping product category codes to product names (assuming prod_cat_info_df contains this information)
prod_cat_info_df.set_index('prod_cat_code', inplace=True)
prod_cat_info = prod_cat_info_df['prod_cat'].to_dict()


In [31]:
# Got product names for collaborative filtering recommendations
collab_recommendations = [prod_cat_info[code] for code in recommendations]
print(f"Collaborative Filtering Recommendations for user_id {user_id}: {collab_recommendations}")



Collaborative Filtering Recommendations for user_id 270351: ['Electronics', 'Footwear', 'Books', 'Bags', 'Home and kitchen', 'Clothing']


In [32]:
# Content-Based Filtering using Cosine Similarity
cosine_sim = cosine_similarity(user_item_matrix)

# Function to recommend items based on similarity
def recommend_items(user_id, num_recommendations=10):
    user_index = user_item_matrix.index.get_loc(user_id)
    similarity_scores = list(enumerate(cosine_sim[user_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_indices = [x[0] for x in similarity_scores[1:num_recommendations+1]]
    return user_item_matrix.index[top_indices]


In [33]:

# Example usage of content-based filtering
similar_users = recommend_items(user_id)
print("Content-Based Similar Users for user_id {}: {}".format(user_id, similar_users))

Content-Based Similar Users for user_id 270351: Index([274483, 272437, 270261, 270496, 274119, 274355, 273906, 274079, 272438,
       275241],
      dtype='int64', name='cust_id')


In [34]:
# Retrieving products interacted with by similar customers
def get_products_for_similar_users(similar_users, final_df, prod_cat_info, top_n=10):
    similar_user_data = final_df[final_df['cust_id'].isin(similar_users)]
    top_products = similar_user_data.groupby('prod_cat_code').sum(numeric_only=True)['total_amt'].sort_values(ascending=False).head(top_n)
    return top_products.index.map(prod_cat_info).tolist()

In [35]:
content_based_recommendations = get_products_for_similar_users(similar_users, final_df, prod_cat_info)
print(f"Content-Based Recommendations for user_id {user_id}: {content_based_recommendations}")



Content-Based Recommendations for user_id 270351: ['Electronics', 'Books', 'Footwear', 'Clothing', 'Bags']


In [36]:
# Evaluation Metrics: Precision, Recall, F1-score, MAP
def evaluate_recommender_system(true_data, predicted_data, top_n=10):
    true_data_binary = true_data > 0
    predicted_data_binary = predicted_data > 0

    precision = precision_score(true_data_binary, predicted_data_binary, average='macro')
    recall = recall_score(true_data_binary, predicted_data_binary, average='macro')
    f1 = f1_score(true_data_binary, predicted_data_binary, average='macro')



In [44]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    """
    return sum([apk(a, p, k) for a, p in zip(actual, predicted)]) / len(actual)


In [45]:
# Spliting data into training and test sets
train_ratio = 0.8
train_size = int(len(user_item_matrix) * train_ratio)
train_data = user_item_matrix.iloc[:train_size]
test_data = user_item_matrix.iloc[train_size:]

In [46]:
# Spliting data into training and test sets
train_ratio = 0.8
train_size = int(len(user_item_matrix) * train_ratio)
train_data = user_item_matrix.iloc[:train_size]
test_data = user_item_matrix.iloc[train_size:]


In [47]:
# Training the model on the training set
train_matrix = svd.fit_transform(train_data)
reconstructed_train_matrix = np.dot(train_matrix, svd.components_)

In [48]:
# Evaluating the model on the test set
test_matrix = svd.transform(test_data)
reconstructed_test_matrix = np.dot(test_matrix, svd.components_)

In [50]:
def evaluate_recommender_system(test_data, reconstructed_test_matrix):
    # Example logic for calculating metrics
    precision = ...  # Calculate precision
    recall = ...     # Calculate recall
    f1 = ...         # Calculate F1 score
    map_score = ...  # Calculate MAP score

    # Return a tuple with the calculated metrics
    return precision, recall, f1, map_score
precision, recall, f1, map_score = evaluate_recommender_system(test_data, reconstructed_test_matrix)


In [51]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"Mean Average Precision (MAP): {map_score}")

Precision: Ellipsis
Recall: Ellipsis
F1-score: Ellipsis
Mean Average Precision (MAP): Ellipsis


In [52]:
# Calculating Recommendation Accuracy
def calculate_accuracy(test_data, reconstructed_test_matrix, top_n=10):
    hits = 0
    total = 0
    for i, user in enumerate(test_data.index):
        actual_items = np.nonzero(test_data.loc[user].values)[0]
        recommended_items = np.argsort(reconstructed_test_matrix[i])[::-1][:top_n]
        hits += len(set(actual_items) & set(recommended_items))
        total += len(actual_items)
    accuracy = hits / total if total > 0 else 0
    return accuracy

accuracy = calculate_accuracy(test_data, reconstructed_test_matrix)
print(f"Recommendation Accuracy: {accuracy}")


Recommendation Accuracy: 1.0


In [53]:
# Checking if accuracy meets the requirement
if accuracy >= 0.8:
    print("The recommender system meets the accuracy requirement of 80% or higher.")
else:
    print("The recommender system does not meet the accuracy requirement of 80% or higher.")


The recommender system meets the accuracy requirement of 80% or higher.
